# i86 - with GPU Tensorflow

In [25]:
import pandas as pd
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']
gpu_hosts = hosts[(hosts.gpu > 0 ) & (hosts.arch == "x86_64")]
gpu_hosts['ip'].to_csv(r'gpu-ip.txt', header=False, index=None, sep=' ')
gpu_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
3,kx-Lenovo-H520g,x86_64,1


In [26]:
%%bash
source ../common/env.sh 
echo "ip|name|id|image|ports|runnning" > containers.csv
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker run --runtime=nvidia --rm nvidia/cuda:10.0-base nvidia-smi
done <gpu-ip.txt

Tue Jan 14 02:51:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.57                 Driver Version: 410.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 105...  Off  | 00000000:01:00.0 Off |                  N/A |
| 25%   35C    P0    N/A /  75W |      0MiB /  4038MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [27]:
%%bash
source ../common/env.sh 
echo "ip|name|id|image|ports|runnning" > containers.csv
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker stop tensorflow
    docker rm tensorflow
    docker run --name tensorflow -d -p 8888:8888 tensorflow/tensorflow:1.12.0-gpu 
    sleep 10
    docker logs tensorflow 2>&1
done <gpu-ip.txt

tensorflow
tensorflow
8c84fec830325682b4933e96a80fc95612fcae5e63e0d99a29126fc91e070ac3
[I 02:51:31.016 NotebookApp] Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
[I 02:51:31.033 NotebookApp] Serving notebooks from local directory: /notebooks
[I 02:51:31.033 NotebookApp] The Jupyter Notebook is running at:
[I 02:51:31.033 NotebookApp] http://(8c84fec83032 or 127.0.0.1):8888/?token=452185a0bb92e3736dfc469f273a2d9fb77e0c5fbc6b4024
[I 02:51:31.033 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 02:51:31.033 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://(8c84fec83032 or 127.0.0.1):8888/?token=452185a0bb92e3736dfc469f273a2d9fb77e0c5fbc6b4024


# GPU on Jetson

In [28]:
jetson_hosts = hosts[(hosts.gpu > 0 ) & (hosts.arch == "aarch64")]
jetson_hosts['ip'].to_csv(r'jetson-ip.txt', header=False, index=None, sep=' ')
jetson_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
12,tegra-ubuntu,aarch64,1


In [29]:
%%bash
source ../common/env.sh 
echo "ip|name|id|image|ports|runnning" > containers.csv
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker run --device=/dev/nvhost-ctrl --device=/dev/nvhost-ctrl-gpu \
    --device=/dev/nvhost-prof-gpu --device=/dev/nvmap --device=/dev/nvhost-gpu \
    --device=/dev/nvhost-as-gpu \
    -v /usr/lib/aarch64-linux-gnu/tegra:/usr/lib/aarch64-linux-gnu/tegra device_query
done <jetson-ip.txt

/cudaSamples/deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "NVIDIA Tegra X2"
  CUDA Driver Version / Runtime Version          9.0 / 9.0
  CUDA Capability Major/Minor version number:    6.2
  Total amount of global memory:                 7854 MBytes (8235802624 bytes)
  ( 2) Multiprocessors, (128) CUDA Cores/MP:     256 CUDA Cores
  GPU Max Clock rate:                            1301 MHz (1.30 GHz)
  Memory Clock rate:                             1600 Mhz
  Memory Bus Width:                              128-bit
  L2 Cache Size:                                 524288 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total am

In [30]:
%%bash
source ../common/env.sh 
echo "ip|name|id|image|ports|runnning" > containers.csv
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker stop tensorflow
    docker rm tensorflow
    docker run --name tensorflow -d -p 8888:8888 tensorflow/tensorflow:1.12.0-gpu 
    sleep 10
    docker logs tensorflow 2>&1
done <jetson-ip.txt

tensorflow
tensorflow
8e96f3606aae68d6073490c07090c84b600b25297737f2916c91d19eaaf4f3a9
standard_init_linux.go:211: exec user process caused "exec format error"


Tensorflow seems to not work on arm cpu, I need to cook a image for it. 


References:
- https://github.com/Technica-Corporation/Tegra-Docker
- https://www.tensorflow.org/guide/gpu
- https://www.tensorflow.org/tfx/serving/docker
- https://github.com/tensorflow/serving
- https://www.tensorflow.org/tutorials/quickstart/beginner
- https://github.com/tensorflow/examples
- https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/dockerfiles
- https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#flexible-upgrade-path